#What is LangGraph


>LangGraph is an orchestration framework for building intelligent, stateful, and multi-step LLM workflows.

It enables advanced features like parallelism, loops, branching, memory, and resumability — making it ideal for agentic and production-grade AI applications.

It models your logic as a graph of nodes (tasks) and edges (routing) instead of a linear chain.

#💡 Why use LangGraph?

>Native branching, loops, and jumps.

>Shared mutable state between nodes.

>Retry/fallback handling via edges.

>Perfect for Agentic AI workflows.

#Basic Idea:
Instead of writing a linear script, you build a graph → each node is a function/tool, and edges define how the workflow moves between nodes.

Example: Hello World LangGraph

```
from langgraph.graph import StateGraph, END

def greet(state):
    return {"message": f"Hello, {state['name']}!"}

graph = StateGraph(dict)
graph.add_node("greet_node", greet)
graph.add_edge("greet_node", END)
graph.set_entry_point("greet_node")

app = graph.compile()
result = app.invoke({"name": "Harsh"})
print(result["message"])
# Output: Hello, Harsh!
```

#LLM Workflows

In LangGraph, an LLM workflow is a sequence of nodes where each node:

>Executes a step (LLM call, API, or tool).

>Updates the state.

>Passes control to the next node based on edges.

💡 Key points:

1.You can mix LLM calls with Python functions.

2.Workflows can be deterministic (fixed path) or dynamic (conditional routing).

#Types of workflow
1.LLM workflows are a step by step process using which we can build complex LLM applications.

2.Each step in a workflow performs a distinct task — such as prompting, reasoning, tool calling, memory access, or decision-making.

3.Workflows can be linear, parallel, branched, or looped, allowing for complex behaviours like retries, multi-agent communication, or tool-augmented reasoning.

4.Common workflows

#Example: Two-step LLM Workflow

```
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def ask_question(state):
    response = llm.invoke("Tell me a fun fact about AI.")
    return {"fact": response.content}

def summarize_fact(state):
    response = llm.invoke(f"Summarize in 10 words: {state['fact']}")
    return {"summary": response.content}

graph = StateGraph(dict)
graph.add_node("ask", ask_question)
graph.add_node("summarize", summarize_fact)

graph.add_edge("ask", "summarize")
graph.add_edge("summarize", END)

graph.set_entry_point("ask")
app = graph.compile()

print(app.invoke({})["summary"])

```

#3. Graphs, Nodes, and Edges
Graph: The whole workflow structure.

Node: A step/function in the workflow.

Edge: A connection between nodes.

1.GenerateTopic

>system generates a relevant UPSC-style essay topic and presents it to the student.

2.CollectEssay

>Student writes and submits the essay based on the generated topic.

3.EvaluateEssay (Parallel Evaluation Block)

>Three evaluation tasks run in parallel:

>>>EvaluateDepth - Analyzes depth of analysis, argument strength, and critical thinking.

>>>EvaluateLanguage - Checks grammar, vocabulary, fluency, and tone.

>>>EvaluateClarity - Assesses coherence, logical flow, and clarity of thought.

4.AggregateResults

>Combines the three scores and generates a total score (e.g., out of 15).

5.ConditionalRouting

>Based on the total score:

>>If score meets threshold → go to ShowSuccess

>>If score is below threshold → go to GiveFeedback

6.GiveFeedback

>Provides targeted suggestions for improvement in weak areas.

7.CollectRevision (optional loop)

>Student resubmits the revised essay.

>Loop back to EvaluateEssay

8.ShowSuccess

>Congratulates the student and ends the flow.



#💡 Two types of edges:

>Normal Edge → Unconditional transition.

>Conditional Edge → Transition based on state values.

#Example: Conditional Edge

```
def check_number(state):
    if state["number"] % 2 == 0:
        return {"status": "even"}
    else:
        return {"status": "odd"}

def even_node(state):
    return {"result": "Number is even"}

def odd_node(state):
    return {"result": "Number is odd"}

graph = StateGraph(dict)
graph.add_node("check", check_number)
graph.add_node("even", even_node)
graph.add_node("odd", odd_node)

graph.add_conditional_edges(
    "check",
    lambda state: state["status"],  # Decision
    {"even": "even", "odd": "odd"}
)

graph.add_edge("even", END)
graph.add_edge("odd", END)
graph.set_entry_point("check")

app = graph.compile()
print(app.invoke({"number": 7})["result"])
```

#4. State
State in LangGraph is:

>A shared dictionary across all nodes.

>Passed and updated incrementally between steps.

>In LangGraph, state is the shared memory that flows through your workflow — it holds all the data being passed between nodes as your graph runs.

```
essay_text: str
topic: str
depth_score: int
language_score: int
clarity_score: int
total_score: int
feedback: Annotated[list[str], add]
evaluation_round: int
```

#💡 Why important?

>Stores intermediate results.

>Controls branching decisions.

>Enables long workflows without losing data.

#Example: Passing State

```
def step_one(state):
    state["x"] = state.get("x", 0) + 5
    return state

def step_two(state):
    state["y"] = state["x"] * 2
    return state

graph = StateGraph(dict)
graph.add_node("one", step_one)
graph.add_node("two", step_two)
graph.add_edge("one", "two")
graph.add_edge("two", END)
graph.set_entry_point("one")

app = graph.compile()
print(app.invoke({"x": 10}))
# {'x': 15, 'y': 30}

```

#5. Reducers

>Reducers in LangGraph define how updates from nodes are applied to the shared state.

>Each key in the state can have its own reducer, which determines whether new data replaces, merges, or adds to the existing value.

>Reducers merge partial updates to the state into the global state.

```

essay_text: str
topic: str
depth_score: int
language_score: int
clarity_score: int
total_score: int
feedback: Annotated[list[str], add]
evaluation_round: int

```

💡

Without reducers → each node replaces state entirely.

With reducers → only updates the necessary keys, keeping the rest.

#Example: Using Reducers
```
from langgraph.graph import MessagesState

def node_a(state: MessagesState):
    return {"messages": ["Step A done"]}

def node_b(state: MessagesState):
    return {"messages": ["Step B done"]}

graph = StateGraph(MessagesState)
graph.add_node("A", node_a)
graph.add_node("B", node_b)
graph.add_edge("A", "B")
graph.add_edge("B", END)
graph.set_entry_point("A")

app = graph.compile()
result = app.invoke({"messages": []})
print(result)
# {'messages': ['Step A done', 'Step B done']}
```

#6. LangGraph Execution Model
Execution steps:

1.Entry point is triggered with an initial state.

2.Node function executes → returns a partial update.

3.Reducer merges update into global state.

4.Edge routing decides next node.

5.Continue until reaching END.

#in depth

1.Graph Definition
You define:

>The state schema

>Nodes (functions that perform tasks)

>Edges (which node connects to which)

2.Compilation
>You call .compile() on the StateGraph.

>This checks the graph structure and prepares it for execution.

3.Invocation
>You run the graph with .invoke(initial_state).
>LangGraph sends the initial state as a message to the entry node(s).

4.Super-Steps Begin
>Execution proceeds in rounds.

5.Message Passing & Node Activation

>The messages are passed to downstream nodes via edges.

>Nodes that receive messages become active for the next round.

6.Halting Condition

Execution stops when:
* No nodes are active, and
* No messages are in transit

💡 Modes:

>Invoke → One-shot execution.

>Stream → Step-by-step execution (good for debugging).

>Async → For concurrent workflows.

#Example: Streaming Execution

```
app = graph.compile()

for step in app.stream({"number": 4}):
    print("Step Output:", step)

```



#7. Visualizing the Graph

LangGraph provides graph.draw() to render the workflow.

#Example Visualization:
```
import matplotlib.pyplot as plt

# Draw and save graph
graph_image = graph.draw()
plt.imshow(graph_image)
plt.axis('off')
plt.show()


#You can also directly save:

graph.draw_png("workflow.png")
```
#✅ With these concepts, you can build, manage, and visualize LangGraph workflows for complex LLM-based applications.

#✅ Summary Table

| Component     | Purpose                  | Example                  |
| ------------- | ------------------------ | ------------------------ |
| **Graph**     | Workflow container       | `StateGraph(dict)`       |
| **Node**      | Step in workflow         | `add_node("name", func)` |
| **Edge**      | Connection between steps | `add_edge("A", "B")`     |
| **State**     | Shared data store        | `state["key"]`           |
| **Reducer**   | Merge updates            | Auto-list append         |
| **Execution** | How graph runs           | `invoke`, `stream`       |
